# Project 2 - Identification of DFAs by Consistent Reduction


## Imports

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve() / "src"))
Path("../outputs").mkdir(parents=True, exist_ok=True)
from dfa_utils import random_dfa, concurrent_composition
from dfa_examples import dfa_simple
from generate_sets import derive_sets
from tree_builder import build_prefix_tree
from lib_const_cover import (
    export_dfa_dot, dot_to_png,
    reduce_negative_set, compute_cover,
    check_cover_redundancy, check_dynamic_consistency, print_cover,
    build_reduced_dfa_from_dynamic_cover, build_initial_cover
)



Using library from: C:\Users\murfe\Desktop\Università\ACCPS_Project\src


## Pipeline 

1) Generation of a reference DFA H through the method dfa_simple().
2) Some utils used to export the obtained DFA in .dot form and, then, to .png

In [13]:
dfa = dfa_simple()
export_dfa_dot(dfa, "../outputs/dfa.dot")   
dot_to_png("../outputs/dfa.dot", "../outputs/dfa.png")

[reduction] .dot saved in ../outputs/dfa.dot
PNG generated: ../outputs/dfa.png


1) Generation of 2 random DFA e to the concurrent composition.
2) Some utils used to export the obtained DFA in .dot form and, then, to .png

In [14]:
dfa1 = random_dfa(n_states=4, alphabet={'a', 'b', 'c', 'd', 'e'}, seed=42)
print("Initial state DFA 1:", dfa1.q0)
export_dfa_dot(dfa1, "../outputs/dfa1.dot")   
dot_to_png("../outputs/dfa1.dot", "../outputs/dfa1.png")

dfa2 = random_dfa(n_states=5, alphabet={'a', 'b','c', 'd', 'e'}, seed=53)
print("Initial state DFA 2:", dfa2.q0)
export_dfa_dot(dfa2, "../outputs/dfa2.dot")   
dot_to_png("../outputs/dfa2.dot", "../outputs/dfa2.png")

composed_dfa = concurrent_composition(dfa1, dfa2)
print("Initial state of the composed DFA:", composed_dfa.q0)
export_dfa_dot(composed_dfa, "../outputs/composed_dfa.dot")
dot_to_png("../outputs/composed_dfa.dot", "../outputs/composed_dfa.png")


Initial state DFA 1: r0
[reduction] .dot saved in ../outputs/dfa1.dot
PNG generated: ../outputs/dfa1.png
Initial state DFA 2: r4
[reduction] .dot saved in ../outputs/dfa2.dot
PNG generated: ../outputs/dfa2.png
Initial state of the composed DFA: ('r0', 'r4')
[reduction] .dot saved in ../outputs/composed_dfa.dot
PNG generated: ../outputs/composed_dfa.png


### Select the DFA that you want to reduce

In [15]:
dfa=dfa #use the reference DFA

#or

#dfa = composed_dfa  #use the composed DFA by 2 random DFAs

1) Generation of the three strings sets A, G and N from the previously generated DFA, with a maximum length (of the strings that belongs to each set) of 6
2) Reduction of the set N through the method reduce_negative_set()

In [16]:
A, G, N = derive_sets(dfa, max_len=6)  
print("A =", sorted(A, key=len))
print("G =", sorted(G, key=len))
N=reduce_negative_set(N)
print("N=", N)


Calculating total word count...
Total words to process: 1092
A = ['abc', 'abcabc']
G = ['a', 'c', 'ac', 'ab', 'abca', 'abcac', 'abcab']
N= {'cc', 'abb', 'acb', 'abcaa', 'acc', 'abcaca', 'abcb', 'abcc', 'aca', 'abcaba', 'abcabb', 'abcacc', 'aba', 'abcacb', 'aa', 'ca', 'b', 'cb'}


1) Construction of the prefix tre acceptor given the three sets A, G and N and the alphabet of the DFA H to reduce

In [17]:
tree = build_prefix_tree(A, G, N, alphabet=dfa.Sigma)
tree.export_dot("../outputs/prefix_tree2.dot")   
dot_to_png("../outputs/prefix_tree2.dot", "../outputs/prefix_tree2.png")

[tree_builder] saved .dot in ../outputs/prefix_tree2.dot
PNG generated: ../outputs/prefix_tree2.png


### Reduction process

1) Cover 0 represents the initial cover given the prefix tree acceptor. This cover X is non redundant
2) Compute the refined cover through the method compute_cover that generate from the initial non-redundant cover a (H-R_C)-consistent cover (a dynamically consistent cover) that is from definition non redundant

In [18]:
cover0 = build_initial_cover(tree)
print("Initial cover:")
print_cover(cover0)
cover_start=cover0.copy()
cover_final = compute_cover(tree, cover_start)
print("Final cover:")
print_cover(cover_final)

Initial cover:
Cover[1] = {, abc, abcabc}
Cover[2] = {, a, ab, abca, abcab, abcac, ac, c}
Cover[3] = {, aa, aba, abb, abcaa, abcaba, abcabb, abcaca, abcacb, abcacc, abcb, abcc, aca, acb, acc, b, ca, cb, cc}
Final cover:
Cover[1] = {abc, abcabc}
Cover[2] = {a, abca}
Cover[3] = {abcac, ac, c}
Cover[4] = {, aa, aba, abb, abcaa, abcaba, abcabb, abcaca, abcacb, abcacc, abcb, abcc, aca, acb, acc, b, ca, cb, cc}
Cover[5] = {ab, abcab}


In [19]:
#Debug final cover
check, problemi = check_dynamic_consistency(cover_final, tree.alphabet)
print("\n--- PRINT FINAL COVER ---")
print_cover(cover_final)

print("\n--- DYNAMIC CONSISTENCY VERIFICATION---")
print(f"Is the cover dinamically consistent? {check}")
if not check:
    print("\nProblem:")
    for pi, sigma, motivo in problemi:
        parole = sorted([n.word for n in pi])
        print(f"Pi: {{{', '.join(parole)}}}, event: {sigma}, reason: {motivo}")

is_ok, problemi = check_cover_redundancy(cover_final)
if is_ok:
    print("The cover has no redundancies")
else:
    print("Redundant cover! Problems found:")
    for i, j in problemi:
        print(f"The cell {j+1} is also in cell {i+1}")



--- PRINT FINAL COVER ---
Cover[1] = {abc, abcabc}
Cover[2] = {a, abca}
Cover[3] = {abcac, ac, c}
Cover[4] = {, aa, aba, abb, abcaa, abcaba, abcabb, abcaca, abcacb, abcacc, abcb, abcc, aca, acb, acc, b, ca, cb, cc}
Cover[5] = {ab, abcab}

--- DYNAMIC CONSISTENCY VERIFICATION---
Is the cover dinamically consistent? True
The cover has no redundancies


### USE OF REDUCED DFA


In [20]:
dfa_red = build_reduced_dfa_from_dynamic_cover(tree, cover_final)
export_dfa_dot(dfa_red, "../outputs/reduced_dfa.dot")   
dot_to_png("../outputs/reduced_dfa.dot", "../outputs/reduced_dfa.png")


Initial is  q3
[reduction] .dot saved in ../outputs/reduced_dfa.dot
PNG generated: ../outputs/reduced_dfa.png
